In [1]:
import Analysis
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
from scipy.optimize import curve_fit, least_squares

In [3]:
def search(data, t):
    seq = data
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max < min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m

In [4]:
def data_partition(data_x,data_y,sections):
    x = list()
    y = list()
    for i in sections:
        i_min = search(data_x,i[0])
        i_max = search(data_x,i[1])+1
        x.extend(data_x[i_min:i_max])
        y.extend(data_y[i_min:i_max])
    return np.array(x), np.array(y)

In [5]:
def func(x,freq,amp,phase_l,height,slope,v0):
    return height + amp * np.sin(freq * x + phase_l) + slope * (x - v0)
    
    
def gen_func(fit):
    def fitting_func(x, phase_r):
        return func(x,fit[0],fit[1],phase_r,fit[3],fit[4],fit[5])
    return fitting_func


def make_fit(popt,popt2,left,right):
    d_phi = ((popt2[0] - popt[2])/np.pi)%2
    return Analysis.Fit([popt[0],popt[1],popt2[0],popt[2],popt[3],popt[4],1,left,right,d_phi,1,popt[5]])

In [6]:
run_2014 = Analysis.generate_2014()
#run_2011 = Analysis.generate_2011()

In [8]:
def convert_fit(out1, out2, xmin, xmax):
    out = list()
    out.append(out1[2])
    out.append(out1[1])
    out.append(out1[3])
    out.append(out2[0])
    out.append(out1[0])
    out.append(out1[4])
    out.append(1.0)
    out.append(xmin)
    out.append(xmax)
    out.append((out1[3]-out2[0])/np.pi%2)
    out.append(1)
    out.append(out1[5])
    return np.array(out)

In [9]:
def regression(filenum,edge_list):

    x_temp, y_temp = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,edge_list)
    
    if len(run_2014.prune_data[filenum].fits) > 0:

        guess = [run_2014.prune_data[filenum].fits[0].freq,run_2014.prune_data[filenum].fits[0].amp,run_2014.prune_data[filenum].fits[0].phase_l,run_2014.prune_data[filenum].fits[0].height,run_2014.prune_data[filenum].fits[0].slope,run_2014.prune_data[filenum].fits[0].v0]
        popt, pcov = curve_fit(func, x_temp, y_temp, p0 = guess)
    
    else:
        popt, pcov = curve_fit(func, x_temp, y_temp)

    return popt

In [32]:
def residual(params, x, y, v0, eps_data):

    model = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[4] * (x - v0)
    
    out = y-model
    
    return out/eps_data

def residual_slip(params, x, y, output, eps_data):

    model = output[0] + output[1] * np.sin(x*output[2] + params[0]) + output[4] * (x - output[5])
    
    out = y-model
    
    return out/eps_data




def regression_scipy(filenum, edge_list, loss = 'linear', eps = 0.1, plot=False, x_scale = 1.0):
    
    x, y = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,edge_list)
    x_min = min(x)
    x_max = max(x)
    
    a = (np.percentile(y,99)-np.percentile(y,1))/2.0
    
    params = []
    bounds = ([],[])
    
    h = np.mean(y)
    params.append(h) #height
    bounds[0].append(h*0.99)
    bounds[1].append(h*1.01)
    
    params.append(a) #amplitude
    bounds[0].append(a*0.8)
    bounds[1].append(a*1.2)
    
    params.append(2*np.pi/10) #frequency
    bounds[0].append(0.6)
    bounds[1].append(0.77)
    
    params.append(0.0) #phase
    bounds[0].append(0.0)
    bounds[1].append(2*np.pi)
    
    params.append(0.0) #slope
    bounds[0].append(-0.005)
    bounds[1].append(0.005)
    
    v0 = (x_min+x_max)/2.0 
    
    
    out = least_squares(residual, x0 = params, bounds=bounds, loss=loss, x_scale = x_scale, args=(x , y, v0, eps))
    
    if plot:
        params = out['x']
        model = params[0] + params[1] * np.sin(x*params[2] + params[3]) + params[4] * (x - v0)
        plt.plot(x,y)
        plt.plot(x,model,c='r')
    
    temp = list(out.x)
    temp.append(v0)
    
    return np.array(temp)

def regression_scipy2(filenum, output, edge_list, loss = 'linear', eps = 0.1, plot=False, x_scale = 1.0):
    
    x, y = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,edge_list)

    
    params = []
    bounds = ([],[])
    
    
    params.append(1.0) #phase
    bounds[0].append(0.0)
    bounds[1].append(2*np.pi)
    
    
    v0 = output[5]
    
    
    out = least_squares(residual_slip, x0 = params, bounds=bounds, loss=loss, x_scale = x_scale, args=(x , y, output, eps))
    
    if plot:
        params = out['x']
        model = output[0] + output[1] * np.sin(x*output[2] + (params[0])) + output[4] * (x - output[5])
        plt.plot(x,y)
        plt.plot(x,model,c='r')
    
    return out.x  

In [11]:
def regression_2(filenum,popt,edge_list):
    slip_func = gen_func(popt)
    x_temp, y_temp = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,edge_list)
    temp = (popt[2]%(2*np.pi))
    guess = temp - 0.25 * np.pi
    popt2, pcov2 = curve_fit(slip_func, x_temp, y_temp,p0=[guess],bounds = ([-2*np.pi],[2*np.pi]))
    return slip_func, popt2

In [12]:
filenum = 7

In [15]:
run_2014.prune_data[filenum].plot()
for i in run_2014.prune_data[filenum].fits:
    i.plot(False)

In [24]:
run_2014.prune_data[filenum].plot()

In [14]:
out = regression_scipy(filenum,[[-13.85,-12.5626],[-12.4042,-8.254],[-6.8238,-6.4471]], plot=True)

In [17]:
out = regression_scipy(filenum,[[-19.8512,-14.2091],[-12.5158,-12.4416],[-8.0718,-7.9194],[-7.58961,-7.45681]], plot=True)

In [45]:
out

array([  1.12256247e+01,   1.79677565e-01,   6.50235207e-01,
         4.41331857e-01,   4.51834740e-03,  -9.97320100e+00])

In [26]:
out2 = regression_scipy2(filenum, out, [[-13.85,-12.5626],[-12.4042,-8.254],[-6.8238,-6.4471]], plot=True,eps=1e-5)

In [27]:
out2

array([  8.57612256e-18])

In [45]:
out2[0] += 0.05

In [46]:
plot_second(out,out2,filenum,[[-13.85,-12.5626],[-12.4042,-8.254],[-6.8238,-6.4471]])

In [33]:
def plot_second(output, params, filenum, edge_list):
    x, y = data_partition(run_2014.prune_data[filenum].x,run_2014.prune_data[filenum].y,edge_list)
    model = output[0] + output[1] * np.sin(x*output[2] + (params[0])) + output[4] * (x - output[5])
    plt.plot(x,y)
    plt.plot(x,model,c='r')

In [47]:
a = convert_fit(out, out2, -19.8512, -6.4471)

In [48]:
A = Analysis.Fit(a)

In [49]:
run_2014.prune_data[filenum].plot()
A.plot(False)

In [51]:
2- A.delta_phi

1.1750704374010845

In [81]:
run_2014.prune_data[filenum].plot_fit(1,False)

In [169]:
run_2014.prune_data[filenum].fits.append(B)

In [85]:
A.output_array()

array([  6.50235161e-01,   1.79677565e-01,   4.41331419e-01,
         2.37005690e+00,   1.12256247e+01,   4.51834806e-03,
         1.00000000e+00,  -1.32758000e+01,  -3.98720000e+00,
         1.38606761e+00,   1.00000000e+00,  -9.97320100e+00])

In [87]:
bf = np.load('52bf.npy')
se = np.load('52se.npy')

In [89]:
hold = list(bf[3])

In [90]:
hold[1] = A.output_array()

In [97]:
run_2014.prune_data[filenum].fits = o

In [96]:
o = [ Analysis.Fit(i) for i in hold]

In [100]:
bf[filenum] = np.array(hold)

In [109]:
for i in run_2014.prune_data[filenum].slip_events(dV=0.04,minsep=3.0):
    d = run_2014.prune_data[filenum]
    y = d.y[search(d.x,i[0])]
    plt.scatter(i[0],y,s=50,c='orange')

In [178]:
bf[filenum] = np.array([i.output_array() for i in run_2014.prune_data[filenum].fits])

In [179]:
bf[filenum]

array([[  8.00157201e-01,   2.45041340e-01,   6.98978566e+00,
          5.85719701e+00,   1.12807537e+01,   0.00000000e+00,
          1.00000000e+00,  -3.00000000e+01,  -2.04705100e+01,
          3.60514164e-01,   3.63600000e+03,  -2.55618100e+01],
       [  6.50235161e-01,   1.79677565e-01,   4.41331419e-01,
          2.37005690e+00,   1.12256247e+01,   4.51834806e-03,
          1.00000000e+00,  -1.32758000e+01,  -3.98720000e+00,
          1.38606761e+00,   1.00000000e+00,  -9.97320100e+00],
       [  6.50235161e-01,   1.79677565e-01,  -2.99999994e-01,
          2.37005690e+00,   1.12256247e+01,   4.51834806e-03,
          1.00000000e+00,  -6.01500000e+00,  -3.98720000e+00,
          1.15009449e+00,   1.00000000e+00,  -9.97320100e+00]])

In [180]:
np.save('52bf',bf)